In [1]:
using CSV, DataFrames, LinearAlgebra, Statistics, Dates, Random
using JuMP, HiGHS
using MathOptInterface
const MOI = MathOptInterface

MathOptInterface

In [2]:
returns_path = joinpath("out", "returns.csv")
tickers_path = joinpath("out", "tickers.csv")
rets_df = CSV.read(returns_path, DataFrame)
tickers = CSV.read(tickers_path, DataFrame).ticker|> collect

50-element Vector{String7}:
 "AAPL"
 "MSFT"
 "GOOGL"
 "AMZN"
 "META"
 "NVDA"
 "AVGO"
 "ORCL"
 "IBM"
 "SAP"
 "JPM"
 "BAC"
 "WFC"
 ⋮
 "NOW"
 "SNOW"
 "AMD"
 "TSM"
 "ASML"
 "AMAT"
 "TSLA"
 "FSLR"
 "ENPH"
 "NEE"
 "PLTR"
 "PANW"

In [3]:
n = length(tickers)
select!(rets_df, [:Date; Symbol.(tickers)])
dates = Date.(rets_df.Date)
Random.seed!(123)
R = Matrix{Float64}(select(rets_df, Not(:Date)))#T daily percent returns of 50 stocks
T = size(R, 1)

443

In [4]:
# The following function will make sure that the covariance matrix of stock returns is positive semidefinite
# which is crucial for the optimizer to work
function psd_clip(Σ; floor::Float64=1e-8)
    vals, vecs= eigen(Symmetric(Σ))
    vals.= max.(vals, floor)
    Σ.= vecs*Diagonal(vals)*vecs'
    Σ.= 0.5.*(Σ.+Σ')
    return Σ
end

psd_clip (generic function with 1 method)

In [5]:
# this function calculates the exponentially weighted mean and covariance of daily percent stock returns,
# putting a higher weight on more recent data
function ew_stats(R::Matrix{Float64}, t::Int; λ::Float64, ridge::Float64=1e-8, floor::Float64=1e-8, shrink::Float64=0.10)
    X = R[1:t, :]
    n = size(X,2)
    Lw = size(X,1)
    pow = λ.^(Lw .-(1:Lw))
    w = (1-λ).*pow
    w./= sum(w)
    μ = vec(X'*w)
    D = X.-(ones(Lw)*permutedims(μ))
    Σ = D'*(D.*w)
    Σ.= 0.5.*(Σ.+Σ')
    if shrink > 0
        z = tr(Σ)/n
        @inbounds for i in 1:n
            Σ[i,i] = (1-shrink)*Σ[i,i]+shrink*z
        end
        @inbounds for j in 1:n, i in 1:n
            if i!=j
                Σ[i,j] = (1-shrink)*Σ[i,j]
            end
        end
    end
    if ridge > 0
        @inbounds for i in 1:n
            Σ[i,i] += ridge
        end
    end
    vals, vecs = eigen(Symmetric(Σ))
    vals.= max.(vals, floor)
    Σ.= vecs*Diagonal(vals)*vecs'
    Σ.= 0.5 .* (Σ.+Σ')
    return μ, Σ
end

ew_stats (generic function with 1 method)

In [6]:
# stochastic simulation of daily percent returns
function simulate_horizon_returns(μ::AbstractVector{<:Real}, Σ::AbstractMatrix{<:Real}; H::Int, S::Int)
    n = length(μ)
    σ2 = diag(Σ)
    μ_log = μ.- 0.5.*σ2
    L = cholesky(Symmetric(Σ), check=false).L

    XH = zeros(Float64, n, S)
    for _ in 1:H
        Z  = randn(n, S)
        Xd = μ_log .+ L * Z
        XH.+= Xd
    end
    Rhor = exp.(XH) .- 1.0
    return Rhor
end

simulate_horizon_returns (generic function with 1 method)

In [7]:
# Rockafellar–Uryasev CVaR optimization with turnover & caps
function optimize_cvar(Rhor::AbstractMatrix{<:Real}, μH::AbstractVector{<:Real}, wprev::AbstractVector{<:Real};
                       alpha::Float64, ret_weight::Float64, c::Float64,
                       cap::Float64, time_limit::Float64)
    n, S = size(Rhor)
    lower = zeros(n)
    upper = cap.*ones(n)
    model = Model(HiGHS.Optimizer); set_silent(model)
    set_optimizer_attribute(model, "time_limit", time_limit)

    @variable(model, lower[i] <= w[i=1:n] <= upper[i])
    @variable(model, s[1:n] >= 0.0) # |w - wprev|
    @variable(model, q) # Free variable for CVaR optimization
    @variable(model, z[1:S] >= 0.0) # tail slacks
    @constraint(model, sum(w) == 1.0)
    @constraint(model, [i=1:n], s[i]>= w[i]-wprev[i])
    @constraint(model, [i=1:n], s[i]>= wprev[i]-w[i])  # these double contrains ensure that s = |w - wprev|
    for sidx in 1:S
        @constraint(model, z[sidx]>= -dot(Rhor[:,sidx],w)-q)
    end
    inv_tail = 1.0/((1-alpha)*S)
    @objective(model, Min, q +(inv_tail*sum(z))-(ret_weight*dot(μH, w))+(c*sum(s)))
    optimize!(model)
    term = termination_status(model); pstat = primal_status(model)
    if (term in (MOI.OPTIMAL, MOI.LOCALLY_SOLVED)) &&
       (pstat in (MOI.FEASIBLE_POINT, MOI.NEARLY_FEASIBLE_POINT))
        return value.(w), "HiGHS_" * string(term)
    else
        return copy(wprev), "FALLBACK_" * string(term)
    end
end

optimize_cvar (generic function with 1 method)

In [8]:
function rebalance_monthdays(mode::Symbol)
    if mode == :quarterly
        return [(7, 1), (10, 1), (1, 1), (4, 1)]
    elseif mode == :monthly
        return [(m, 1) for m in 1:12]
    else
        error("Please use monthly or quarterly rebalance")
    end
end

rebalance_monthdays (generic function with 1 method)

In [9]:
function make_rebalance_dates(dates::Vector{Date}; mode::Symbol=:quarterly, start_at::Date=dates[1])
    md = rebalance_monthdays(mode)
    y0, y1 = year(start_at), year(dates[end])
    rebalances = Date[]
    for y in y0:y1
        for (m,d) in md
            if Dates.daysinmonth(Date(y, max(1,min(12,m)),1)) >= d
                push!(rebalances, Date(y,m,d))
            end
        end
    end
    sort!(rebalances)
    filter!(a -> a >= start_at && a <= dates[end], rebalances)
    return rebalances
end

make_rebalance_dates (generic function with 1 method)

In [10]:
function map_to_trading(dates::Vector{Date}, rebalances::Vector{Date})
    idxs = Int[]
    for a in rebalances
        k = searchsortedfirst(dates, a)
        if k <= length(dates); push!(idxs, k); end
    end
    unique!(idxs)
    return idxs
end

map_to_trading (generic function with 1 method)

In [11]:
function graph_quantiles!(graph_df::DataFrame, μ::Vector{Float64}, Σ::Matrix{Float64}, w::Vector{Float64};
                        dates::Vector{Date}, t::Int, H::Int, S::Int, PV0::Float64)
    n = length(w)
    σ2 = diag(Σ)
    μ_log = μ .- 0.5 .*σ2
    L = cholesky(Symmetric(Σ), check=false).L
    PV = fill(PV0, S)
    Y  = zeros(Float64, S)

    function push_row!(h::Int)
        q  = quantile(PV, [0.05, 0.25, 0.50, 0.75, 0.95])
        μY = mean(Y); σY = std(Y)
        μ1lo = PV0 * exp(μY - 1σY); μ1hi = PV0 * exp(μY + 1σY)
        μ2lo = PV0 * exp(μY - 2σY); μ2hi = PV0 * exp(μY + 2σY)
        μ3lo = PV0 * exp(μY - 3σY); μ3hi = PV0 * exp(μY + 3σY)
        push!(graph_df, (
            rebalance_date = dates[t],
            h = h,
            date = dates[min(t+h, length(dates))],
            pv_rebalance= PV0,
            pv_q05=q[1], pv_q25=q[2], pv_q50=q[3], pv_q75=q[4], pv_q95=q[5],
            pv_mu_1s_lo=μ1lo, pv_mu_1s_hi=μ1hi,
            pv_mu_2s_lo=μ2lo, pv_mu_2s_hi=μ2hi,
            pv_mu_3s_lo=μ3lo, pv_mu_3s_hi=μ3hi,
        ))
    end

    push_row!(0)
    for _h in 1:H
        Z = randn(n, S)
        Xd = μ_log .+ L*Z
        Rd = exp.(Xd) .-1.0
        r = vec(w'*Rd)
        PV .*= (1 .+r)
        Y .+= log.(1 .+r)
        push_row!(_h)
    end
end

graph_quantiles! (generic function with 1 method)

In [12]:
invest_start_date = Date("2024-07-01") # date of portfolio creation
rebalance_mode= :quarterly # :monthly or :quarterly

# EWMA parameters
half_life = 60.0
λ_ew = 2.0^(-1.0 / half_life)
ridge_gamma = 1e-8
eig_floor = 1e-8
delta_shrink = 0.10

# Risk/return + trade costs
alpha = 0.95 # CVaR tail level
ret_weight = 1.0 # return reward
c_turnover = 0.005 # L1 turnover penalty
exec_cost_bps = 10.0 # per $ traded
exec_cost_rate = exec_cost_bps/10000

cap = 0.20 # maximum proportion of portfolio allowed in a single stock.

# Simulation + solver
num_of_scenarios = 2000 # used for optimizer & graphing
time_limit_s = 300.0 # time limit in seconds to solve one rebalance

300.0

In [13]:
start_idx = findfirst(dates .>= invest_start_date)
@assert start_idx !== nothing "No data on/after chosen portfolio creation date"

rebalance_cals = make_rebalance_dates(dates; mode=rebalance_mode, start_at=invest_start_date)
rebalance_idxs = map_to_trading(dates, rebalance_cals)
idxs = vcat(rebalance_idxs, T) # rebalance idx's plus last idx

8-element Vector{Int64}:
  63
 127
 191
 251
 313
 377
 441
 443

In [14]:
rebalance_cals

7-element Vector{Date}:
 2024-07-01
 2024-10-01
 2025-01-01
 2025-04-01
 2025-07-01
 2025-10-01
 2026-01-01

In [15]:
rebalance_idxs

7-element Vector{Int64}:
  63
 127
 191
 251
 313
 377
 441

In [16]:
w_prev = fill(1.0/n, n)
port_val = 1.0 # starting portfolio value set to 1.0
last_idx = start_idx

63

In [17]:
# daily portfolio series (for plots)
daily_dates = Date[]
daily_pv = Float64[]
push!(daily_dates, dates[last_idx]); push!(daily_pv, port_val)

1-element Vector{Float64}:
 1.0

In [18]:
vals = Float64[]; reb_dates = Date[]; turnover_costs = Float64[]
est_h_ret = Float64[]; est_h_std = Float64[]
statuses = String[]; weights_snap = Vector{Vector{Float64}}()
top3_tickers = String[]; top3_weights = Vector{Tuple{Float64,Float64,Float64}}()

Tuple{Float64, Float64, Float64}[]

In [19]:
graph_df = DataFrame(
    rebalance_date = Date[], h = Int[], date = Date[], pv_rebalance = Float64[],
    pv_q05=Float64[], pv_q25=Float64[], pv_q50=Float64[], pv_q75=Float64[], pv_q95=Float64[],
    pv_mu_1s_lo=Float64[], pv_mu_1s_hi=Float64[],
    pv_mu_2s_lo=Float64[], pv_mu_2s_hi=Float64[],
    pv_mu_3s_lo=Float64[], pv_mu_3s_hi=Float64[],
)

Row,rebalance_date,h,date,pv_rebalance,pv_q05,pv_q25,pv_q50,pv_q75,pv_q95,pv_mu_1s_lo,pv_mu_1s_hi,pv_mu_2s_lo,pv_mu_2s_hi,pv_mu_3s_lo,pv_mu_3s_hi
,Date,Int64,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [20]:
for (j, t) in enumerate(idxs)
    # calculating Profit and loss of previous weights up to this date t
    if t > last_idx
        seg = R[(last_idx+1):t, :]'
        seg_port = vec(w_prev' * seg)
        for k in 1:length(seg_port)
            port_val *= (1 + seg_port[k])
            push!(daily_dates, dates[last_idx + k])
            push!(daily_pv, port_val)
        end
    end

    # no rebalance on last day unless its a rebalance day
    is_final = (j == length(idxs))
    if is_final
        break
    end

    println("Reb ", dates[t], " (start)")

    # Quarter EWMA stats ending at this rebalance date
    μ_t, Σ_t = ew_stats(R, t; λ=λ_ew, ridge=ridge_gamma, floor=eig_floor, shrink=delta_shrink)

    # Get horizon to next rebalance (or today if last real rebalance)
    H_t = max(1, idxs[j+1]-t)

    # asset returns over horrizon by daily stochastic simulation
    Rhor = simulate_horizon_returns(μ_t, Σ_t; H=H_t, S=num_of_scenarios)
    μH = vec(mean(Rhor, dims=2))

    # get new weights
    w_t, status = optimize_cvar(Rhor, μH, w_prev;
                                alpha=alpha, ret_weight=ret_weight, c=c_turnover,
                                cap=cap, time_limit=time_limit_s)

    # Execution reflected in both PV and today's daily value
    turnover = sum(abs.(w_t .- w_prev))
    cost = exec_cost_rate * turnover * port_val
    port_val -= cost
    daily_pv[end] = port_val # same day reflects execution cost

    # stats upto end of quarter
    Rhor_c = Rhor .-μH
    ΣH = (Rhor_c *Rhor_c')/(size(Rhor,2)-1)
    ΣH .= 0.5 .*(ΣH.+ΣH'); psd_clip(ΣH; floor=eig_floor)
    μp = dot(μH, w_t)
    σp = sqrt(dot(w_t, ΣH * w_t))

    # graph rows anchored at current wealth
    graph_quantiles!(graph_df, μ_t, Σ_t, w_t; dates=dates, t=t, H=H_t, S= num_of_scenarios, PV0=port_val)

    # Log row (real rebalance only)
    push!(vals, port_val); push!(reb_dates, dates[t])
    push!(turnover_costs, cost)
    push!(est_h_ret, μp); push!(est_h_std, σp)
    push!(statuses, status); push!(weights_snap, copy(w_t))

    ord = sortperm(w_t, rev=true); tops = ord[1:3]
    push!(top3_tickers, string(join(tickers[tops], ",")))
    push!(top3_weights, (w_t[tops[1]], w_t[tops[2]], w_t[tops[3]]))

    println("Reb ", dates[t], " status=", status,
            " μ≈", round(100*μp; digits=2), "% over horizon",
            " σ≈", round(100*σp; digits=2), "% over horizon",
            " turnover=", round(100*turnover; digits=2), "% PV=", round(port_val; digits=4))

    w_prev = w_t; last_idx = t
end

Reb 2024-07-01 (start)
Reb 2024-07-01 status=HiGHS_OPTIMAL μ≈32.26% over horizon σ≈12.7% over horizon turnover=172.0% PV=0.9983
Reb 2024-10-01 (start)
Reb 2024-10-01 status=HiGHS_OPTIMAL μ≈25.02% over horizon σ≈12.6% over horizon turnover=181.53% PV=1.0175
Reb 2025-01-02 (start)
Reb 2025-01-02 status=HiGHS_OPTIMAL μ≈35.37% over horizon σ≈23.59% over horizon turnover=101.78% PV=1.2225
Reb 2025-04-01 (start)
2025-04-01 status=HiGHS_OPTIMAL μ≈15.38% over horizon σ≈12.38% over horizon turnover=160.0% PV=1.0675
Reb 2025-07-01 (start)
Reb 2025-07-01 status=HiGHS_OPTIMAL μ≈28.07% over horizon σ≈23.43% over horizon turnover=135.86% PV=1.1936
Reb 2025-10-01 (start)
2025-10-01 status=HiGHS_OPTIMAL μ≈30.77% over horizon σ≈20.73% over horizon turnover=108.89% PV=1.4485
Reb 2026-01-02 (start)
Reb 2026-01-02 status=HiGHS_OPTIMAL μ≈0.27% over horizon σ≈0.97% over horizon turnover=132.25% PV=1.4646


In [21]:
summary_df = DataFrame(
    Date=reb_dates,
    portfolio_value=vals,
    est_horizon_ret=est_h_ret,
    est_horizon_std=est_h_std,
    turnover_cost=turnover_costs,
    status=statuses,
    top3=top3_tickers,
    top3_w1=[tw[1] for tw in top3_weights],
    top3_w2=[tw[2] for tw in top3_weights],
    top3_w3=[tw[3] for tw in top3_weights],
)
CSV.write(joinpath("out", "multi_period_rebalance_summary.csv"), summary_df)

"out\\multi_period_rebalance_summary.csv"

In [22]:
summary_df

Row,Date,portfolio_value,est_horizon_ret,est_horizon_std,turnover_cost,status,top3,top3_w1,top3_w2,top3_w3
,Date,Float64,Float64,Float64,Float64,String,String,Float64,Float64,Float64
1,2024-07-01,0.99828,0.322642,0.12695,0.00172,HiGHS_OPTIMAL,"AAPL,NVDA,LLY",0.2,0.2,0.2
2,2024-10-01,1.01747,0.250247,0.126036,0.0018504,HiGHS_OPTIMAL,"ORCL,WMT,NEE",0.2,0.2,0.2
3,2025-01-02,1.22247,0.353733,0.235906,0.00124546,HiGHS_OPTIMAL,"AVGO,WMT,TSLA",0.2,0.2,0.2
4,2025-04-01,1.06747,0.153827,0.123821,0.00171069,HiGHS_OPTIMAL,"BRK-B,KO,ABT",0.2,0.2,0.2
5,2025-07-01,1.1936,0.280713,0.234324,0.00162386,HiGHS_OPTIMAL,"ORCL,IBM,PLTR",0.2,0.2,0.2
6,2025-10-01,1.44847,0.307651,0.207269,0.0015789,HiGHS_OPTIMAL,"GOOGL,PLTR,ORCL",0.2,0.2,0.153931
7,2026-01-02,1.46461,0.00269885,0.00973154,0.00193953,HiGHS_OPTIMAL,"WMT,GOOGL,KO",0.161979,0.145274,0.131588


In [23]:
weights_df = DataFrame(Date=reb_dates)
for j in 1:n
    weights_df[!, Symbol(tickers[j])] = [w[j] for w in weights_snap]
end
CSV.write(joinpath("out", "multi_period_weights.csv"), weights_df)

"out\\multi_period_weights.csv"

In [24]:
weights_df

Row,Date,AAPL,MSFT,GOOGL,AMZN,META,NVDA,AVGO,ORCL,IBM,SAP,JPM,BAC,WFC,MS,GS,BLK,V,MA,BRK-B,KO,PG,PEP,WMT,COST,HD,MCD,DIS,NKE,GE,UNH,LLY,PFE,MRK,ABT,TMO,AMGN,ADBE,CRM,NOW,SNOW,AMD,TSM,ASML,AMAT,TSLA,FSLR,ENPH,NEE,PLTR,PANW
,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2024-07-01,0.2,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.106938,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.139317,0.0360902,0.0,0.0,0.0,0.117655
2,2024-10-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.107667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0911171,0.00121579,0.0,0.2,0.2,0.0
3,2025-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0115054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.177916,0.0105785,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.2,0.0
4,2025-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.119796,0.0298195,0.0,0.0,0.0,0.0,0.0,0.0,0.0286908,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0207989,0.0,0.0,0.000895146,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0
5,2025-07-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0410333,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0859974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.116712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.156257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0
6,2025-10-01,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.153931,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.101641,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.075644,0.0855246,0.0,0.141811,0.0414484,0.0,0.0,0.2,0.0
7,2026-01-02,0.0,0.0,0.145274,0.0,0.0,0.0,0.0,0.0106771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00166215,0.0,0.131077,0.131588,0.061302,0.0650987,0.161979,0.0,0.0,0.0229731,0.0,0.0,0.0681363,0.0,0.016057,0.0,0.0557922,0.011968,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0739304,0.0191935,0.0,0.0,0.0209024,0.0,0.00175573,0.000632827,0.0


In [25]:
# Daily portfolio values (from first rebalance day to "today")
daily_df = DataFrame(Date=daily_dates, portfolio_value=daily_pv)
CSV.write(joinpath("out", "portfolio_daily.csv"), daily_df)

"out\\portfolio_daily.csv"

In [26]:
daily_df

Row,Date,portfolio_value
,Date,Float64
1,2024-07-01,0.99828
2,2024-07-02,1.01176
3,2024-07-03,1.03264
4,2024-07-05,1.0404
5,2024-07-08,1.04596
6,2024-07-09,1.05799
7,2024-07-10,1.07042
8,2024-07-11,1.03683
9,2024-07-12,1.05227


In [27]:
# data for graphing in post optimization analysis
CSV.write(joinpath("out", "graph.csv"), graph_df)

"out\\graph.csv"